In [13]:
# バイナリデータ扱うモジュール
import struct
from PIL import Image, ImageEnhance
import glob, os

# 出力ディレクトリ
outdir = "png-etl1/"
if not os.path.exists(outdir) : os.mkdir(outdir)

# ETL1ディレクトリ以下のファイルを処理する
files = glob.glob("ETL1/*")
for fname in files :
    # 情報ファイルは飛ばす
    if fname == "ETL1/ETL1INFO" : continue
    print(fname)
    # ETL1のデータファイルを開く（バイナリ読み込み）
    f = open(fname, 'rb')
    # ファイルの先頭にポインタセット
    f.seek(0)
    while True : 
        # メタデータ＋画像の組みを一つずつ読む（各データは2052バイトの固定長）
        s = f.read(2052)
        if not s : break
        # バイナリデータをPythonで読めるように抽出
        r = struct.unpack('>H2sH6BI4H4B4x2016s4x', s)
        code_ascii = r[1] #s(char)
        code_jis = r[3]
        # 画像データを取り出す　fなのでfloat bit point型
        # r[18]：？
        # PIL.Image.frombytes(mode, size, data, decoder_name='raw', *args)　　args decodeのパラメータ
        iF = Image.frombytes('F', (64,63), r[18] , 'bit', 4)
        #グレースケールに変換
        iP = iF.convert('L')
        # 画像を鮮明にして保存
        dir = outdir + "/" + str(code_jis)
        if not os.path.exists(dir) : os.mkdir(dir)
        # r[0]:H(Integer),r[2]:s(char)
        # formatあとの引数を順位に0,1,2のとこに入れる
        fn = "{0:02x}-{1:02x}{2:04x}.png".format(code_jis, r[0],r[2])
        fullpath = dir + "/" + fn
        if os.path.exists(fullpath) : continue
        # 明るさ調整
        enhancer = ImageEnhance.Brightness(iP)
        # 鮮明に 1だと元画像と同じ
        iE = enhancer.enhance(16)
        iE.save(fullpath, 'PNG')
print("OK")

ETL1/ETL1C_08


AttributeError: 'Image' object has no attribute 'shape'